In [1]:
import init

import findspark
findspark.init()

import os
import pandas as pd
import numpy as np
from pyspark.sql import *
from pyspark import *
import pyspark.sql.functions as f
from pyspark.sql.types import *
import time
from pyrecdp.data_processor import *
from pyrecdp.encoder import *
from pyrecdp.utils import *

spark = SparkSession.builder.master('yarn')\
        .appName("Recsys2021_data_process")\
        .getOrCreate()
current_path = "/recsys2021_0608_processed/"
path_prefix = "hdfs://"
dicts_folder = "recsys_dicts/"

In [7]:
from pyspark.ml.feature import *
train_data_path= "/recsys2021_0608_processed/tweet_text_processed_train_with_fold"
train = spark.read.parquet("%s/" % (train_data_path))
df = train.select('engaged_with_user_id', 'engaged_with_user_follower_count')\
          .groupby('engaged_with_user_id')\
          .agg(f.max(f.col('engaged_with_user_follower_count')))\
          .withColumnRenamed('max(engaged_with_user_follower_count)', 'engaged_with_user_follower_count')
print("after agg, total number of engaged_with_user_id is %d" % df.count())
df.show()
qd = QuantileDiscretizer(numBuckets=5, inputCol="engaged_with_user_follower_count", outputCol='user_quantile')
qdmodel = qd.fit(df)
display(qdmodel.getSplits())

after agg, total number of engaged_with_user_id is 21894172
+--------------------+--------------------------------+
|engaged_with_user_id|engaged_with_user_follower_count|
+--------------------+--------------------------------+
|             1757710|                           25273|
|              310514|                             979|
|                3749|                         1028270|
|               16574|                         1163959|
|             3968160|                             686|
|             1804093|                            2472|
|              127109|                           31716|
|               15790|                            1623|
|            17421974|                             540|
|            16154717|                             384|
|               35361|                           24650|
|              346809|                           51794|
|               42635|                          124713|
|             5136054|                      

[-inf, 102.0, 237.0, 482.0, 1164.0, inf]